# Factorial Experiments

While at its core, the multiple linear regression that goes along with the Design of Experiments section is the same as we saw earlier, there are some nuances related to building the models and interpreting the output. 

Let's take a look at the example from slide 45 of the Design of Experiments lectures (week 8). In this case, we had a 2 factor factorial. 

In [1]:
import numpy as np
from statsmodels.formula.api import ols

A = np.array([-1, 1, -1, 1])
B = np.array([-1, -1, 1, 1])
y = [69, 60, 64, 53]


In the least squares GutHub tutorial we built this model by manually typing out the $X$ matrix and then fitting the model. There is an easier way to do this. 

In [2]:
model=ols("y~A*B", {'y': y, 'A': A, 'B': B})

Creating the model in the above manner automatically generates the 2 factor interaction. Let's see what the results of the model are: 

In [3]:
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 07 Nov 2021   Prob (F-statistic):                nan
Time:                        10:30:50   Log-Likelihood:                 126.02
No. Observations:                   4   AIC:                            -244.0
Df Residuals:                       0   BIC:                            -246.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     61.5000        inf          0        n

c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\linear_model.py:1650: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
c:\use

The A:B interaction is automatically included and we get the same coefficients as we calculated in the lectures! 

Note that the Python output includes several warnings reminding us that we don't have the necessary degrees of freedom to calculate the standard error and therefore the confidence intervals of the model coefficients (among other values). The corresponding values in the table are some variation of undefined (either inf or nan).

## Fractional Factorials

One issue that complicates things in Python is when we do fractional experiments. Let's look at an example where we have three factors but we're running a half factorial. To generate the half factorial we define C = AB. A and B are the same in this case as defined above. 

In [4]:
C = A*B
y = [30, 6, 4, 4]
print(C)

[ 1 -1 -1  1]


Now let's build the model and see our results. 

In [5]:
model=ols("y~A*B*C", {'y': y, 'A': A, 'B': B, 'C':C})
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 07 Nov 2021   Prob (F-statistic):                nan
Time:                        10:30:50   Log-Likelihood:                 127.29
No. Observations:                   4   AIC:                            -246.6
Df Residuals:                       0   BIC:                            -249.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.5000        inf          0        n

c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\linear_model.py:1650: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


The results look somewhat strange. How are we estimating 8 coefficients (3 main effects + 3 two factor interactions + 1 three factor interaction + intercept) if we only have 4 data points? The reason for this is that several of the effects are confounded! Python reports the total coefficient values, split among all the different terms that are confounded!

We defined in the lectures that the defining relationship in this case is $I=ABC$. Based on this defining relationship we determined that A is alisased with BC, B is aliased with AC, C is aliased with AB (by design), and that the intercept is aliased with ABC. The coefficients are therefore represented as follows: 

$b_0 + b_{ABC} = \hat{\beta_0}$ 

$b_A + b_{BC} = \hat{\beta_A}$

$b_B + b_{AC} = \hat{\beta_B}$

$b_C + b_{AB} = \hat{\beta_C}$

The linear model can therefore be written as follows: $y_i = \hat{\beta_0} + \hat{\beta_A}x_a + \hat{\beta_B}x_b + \hat{\beta_C}x_c = (5.5 + 5.5) + (-3 -3)x_a + (-3.5 -3.5)x_b + (3 +3)x_c = 11 -6x_a -7x_b + 6x_c$

Note that in this case each coefficient was created by two aliased terms. If more terms are aliased, Python will split the coefficient value between all those terms. 

Another way to do this is to simply include the terms that you want in the model definition. In the above example we know that for a half $2^3$ factorial we can only estimate the three main effects (confounded with the two factor interactions) and the intercept (confounded with the three factor interaction) for a total of 4 coefficients. If we want the value of the coefficients directly, instead of using the `*` notation to generate the interaction effects automatically we can just include the three terms in the model that we want. 

In [6]:
model=ols("y~A+B+C", {'y': y, 'A': A, 'B': B, 'C':C})
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 07 Nov 2021   Prob (F-statistic):                nan
Time:                        10:30:50   Log-Likelihood:                 125.65
No. Observations:                   4   AIC:                            -243.3
Df Residuals:                       0   BIC:                            -245.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.0000        inf          0        n

c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\users\degidal\appdata\local\programs\python\python38\lib\site-packages\statsmodels\regression\linear_model.py:1650: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
c:\use

The coefficients we get above are the same as when we included the interaction terms and combine the confounded ones manually. 

To summarize, in the OLS function the `*` operator can be used to automatically generate the interaction terms between the different variables. That being said, care needs to be taken when interpreting the output. If variables are confounded, Python will split the actual coefficient value evenly between all the confounded terms. The coefficient can  then be recovered by summing up the coefficient for the confounded term. Altenately this can be accounted for manually during the model definition, by only including the desired columns. The reported coefficient value in this case is still confounded, but only a single value is reported based on how we defined the model. 